In [10]:
import rioxarray as rx
import json
import xarray as xr

In [2]:
domain = 'OR'

FLAG_MET=1          #set to 1 if you want to convert meteo forcing (most common task)
FLAG_MET_EXTRA=0    #set to 1 if you want incoming LWR & SWR
FLAG_DEM_LC=0       #set to 1 if you want to convert DEM and
                    #Land Cover (typically only needed once)
FLAG_DEM_EXTRA=0    #set to 1 if you want the 'extra' lon/lat grids
                    #for large domains (typically only needed once)
FLAG_LR=0           #set to 1 if you want to compute lapse rates / adjustment
                    #factors from PRISM data (typically only needed once)

In [25]:
#CSO data path
CSOpath = 'cso-data.geojson'

#path to CSO domain json
with open('/nfs/attic/dfh/Aragon2/CSOdata/CSO_domains.json') as f:
    domains = json.load(f)

#start date    
st = domains[domain]['st']
#end date
ed = domains[domain]['ed']

#paths 
elev = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_elev.tif'
lwr = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_lwr.tif'
prec = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_prec.tif'
spechum = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_spechum.tif'
surfpres = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_surfpres.tif'
swr = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_swr.tif'
tair = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_tair.tif'
uwind = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_uwind.tif'
vwind = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/cfsv2_'+st+'_'+ed+'_vwind.tif'
dem = 'DEM_'+domain+'.tif'
lc = 'NLCD2016_'+domain+'.tif'

#output filename
outfile = 'mm_'+domain+st[:4]+'-'+ed[:4]+'.dat'

#start information
styr = st[:4]
stmo = st[5:7]
stdy = st[8:10]
pointsperday = 4
starthr = 0

In [23]:
#In this section, we will deal with the climate data files load files. 
#All the Rs should be the same, but I read them in as different references anyway.

if FLAG_MET

    tmpfile=[filename '_elev.tif']; # m
    [Z,R_z]=geotiffread(fullfile(pathname,tmpfile));

    tmpfile=[filename '_prec.tif'];  # kg / s / m^2 (this is a precip rate)
    [Pr,R_pr]=geotiffread(fullfile(pathname,tmpfile));

    tmpfile=[filename '_spechum.tif']; # unitless
    [H,R_h]=geotiffread(fullfile(pathname,tmpfile));

    tmpfile=[filename '_surfpres.tif'];  # Pascals
    [P,R_p]=geotiffread(fullfile(pathname,tmpfile));

    tmpfile=[filename '_tair.tif']; # Kelvin
    [T,R_t]=geotiffread(fullfile(pathname,tmpfile));

    tmpfile=[filename '_uwind.tif'];  # m/s
    [U,R_u]=geotiffread(fullfile(pathname,tmpfile));

    tmpfile=[filename '_vwind.tif'];  # m/s
    [V,R_v]=geotiffread(fullfile(pathname,tmpfile));

    %compute number of grid points and time steps from size of 3d matrix
    [y,x,t]=size(Pr);
    gridpts=x*y;
    tsteps=t;

    %create y m d h vectors
    initialdatenum=datenum(startyear,startmonth,startday,starthour,0,0);
    datenums=initialdatenum+(1/pointsperday)*[0:1:tsteps-1]';
    [y,m,d,h,mins,sec]=datevec(datenums); %dont care about min / sec

    %create ID numbers for the grid points
    ID=1e6+[1:1:gridpts]';

    %create matrices of x and y values
    info=geotiffinfo(fullfile(pathname,tmpfile));
    [X,Y]=pixcenters(info,'makegrid');

    %elevation is static (doesn't change with time)
    elev=Z(:,:,1);

    %find number of grid points with <0 elevation. Note: this is related to the
    %subroutine met_data_check in the preprocess_code.f. that subroutine seems
    %to suggest that negative elevations are ok (say, death valley). But, the
    %code itself checks for negative elevations and stops execution is any
    %negatives are found. So, here, I scan for negative depths (say, a weather
    %analysis grid point over the ocean, where bathymetric depths are below sea
    %level) and I remove those points from the output that I create.
    I=find(elev(:)<=0);
    numnegz=length(I); %number of points with negative depths.
    validgridpts=gridpts-numnegz;

    %we are now ready to begin our main loop over the time steps.
    fid=fopen(fullfile(pathname,outfilename),'w');

    %define main format string
    fmt='%5d %3d %3d %6.3f %9d %12.1f %12.1f %8.1f %9.2f %9.2f %9.2f %9.2f %9.2f\n';
    for j=1:tsteps
        %first we write the number of grid points
        fprintf(fid,'%6d\n',validgridpts);

        %prep data matrix for this time step. First, grab the jth time slice
        Prtmp=Pr(:,:,j);
        Htmp=H(:,:,j);
        Ptmp=P(:,:,j);
        Ttmp=T(:,:,j);
        Utmp=U(:,:,j);
        Vtmp=V(:,:,j);

        %convert precip rate to precip DEPTH (mm) during time interval
        Prtmp=Prtmp*24*3600/pointsperday;

        %convert specific hum. to RH from Clausius-Clapeyron. T is still in K
        RHtmp=0.263*Ptmp.*Htmp.*(exp(17.67*(Ttmp-273.16)./(Ttmp-29.65))).^(-1);

        %compute wind speed
        SPDtmp=sqrt(Utmp.^2+Vtmp.^2);

        %compute wind direction. 0-360, with 0 being true north! 90 east, etc.
        DIRtmp=atan2d(Utmp,Vtmp);
        K=find(DIRtmp>=180);
        J=find(DIRtmp<180);
        DIRtmp(K)=DIRtmp(K)-180;
        DIRtmp(J)=DIRtmp(J)+180;

        %put T in C
        Ttmp=Ttmp-273.16;

        data=[y(j)*ones(size(ID)) m(j)*ones(size(ID)) d(j)*ones(size(ID)) ...
            h(j)*ones(size(ID)) ID X(:) Y(:) elev(:) Ttmp(:) RHtmp(:) SPDtmp(:) ...
            DIRtmp(:) Prtmp(:)];

        %remove data at points with neg elevations
        data(I,:)=[];

        fprintf(fid,fmt,data');

        %display progress to screen.
        tmp=round(tsteps/10);
        if mod(j,tmp)==0
            disp(['conversion is ' num2str(j/tsteps*100) ' % done']);
        end
    end
    fclose(fid);
end


'01'

In [6]:
el = rx.open_rasterio(elev)
el

<xarray.DataArray (band: 7424, y: 9, x: 9)>
[601344 values with dtype=float32]
Coordinates:
  * band         (band) int64 1 2 3 4 5 6 7 ... 7419 7420 7421 7422 7423 7424
  * y            (y) float64 4.984e+06 4.962e+06 ... 4.828e+06 4.806e+06
  * x            (x) float64 5.217e+05 5.439e+05 ... 6.771e+05 6.993e+05
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    grid_mapping:  spatial_ref

In [11]:
el.sel(band=1)

<xarray.DataArray (y: 9, x: 9)>
array([[ 542.56 ,  542.56 ,  848.224, 1145.44 , 1145.44 ,  946.912,  759.264,
         773.216,  913.76 ],
       [ 597.984,  597.984,  894.944, 1176.544, 1176.544,  982.624,  818.784,
         942.048, 1087.584],
       [ 578.656,  578.656,  856.032, 1247.968, 1247.968, 1058.912,  883.552,
        1054.688, 1270.24 ],
       [ 244.192,  576.096, 1160.928, 1382.752, 1390.176,  982.752,  926.56 ,
        1062.88 , 1411.68 ],
       [ 301.024,  624.224, 1239.776, 1501.28 , 1559.008, 1220.96 , 1130.976,
        1179.488, 1349.984],
       [ 301.024,  624.224, 1239.776, 1524.32 , 1602.656, 1457.76 , 1414.88 ,
        1421.024, 1404.256],
       [ 411.232,  761.696, 1300.448, 1524.32 , 1602.656, 1513.824, 1524.96 ,
        1531.744, 1464.544],
       [ 556.512,  972.256, 1383.776, 1506.144, 1540.96 , 1513.824, 1472.352,
        1453.792, 1409.12 ],
       [ 609.76 , 1134.688, 1518.048, 1518.048, 1536.736, 1478.88 , 1472.352,
        1416.544, 1341.792]], dtype=float32)
Coordinates:
    band         int64 1
  * y            (y) float64 4.984e+06 4.962e+06 ... 4.828e+06 4.806e+06
  * x            (x) float64 5.217e+05 5.439e+05 ... 6.771e+05 6.993e+05
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    grid_mapping:  spatial_ref

In [12]:
el[1,:,:]

<xarray.DataArray (y: 9, x: 9)>
array([[ 542.56 ,  542.56 ,  848.224, 1145.44 , 1145.44 ,  946.912,  759.264,
         773.216,  913.76 ],
       [ 597.984,  597.984,  894.944, 1176.544, 1176.544,  982.624,  818.784,
         942.048, 1087.584],
       [ 578.656,  578.656,  856.032, 1247.968, 1247.968, 1058.912,  883.552,
        1054.688, 1270.24 ],
       [ 244.192,  576.096, 1160.928, 1382.752, 1390.176,  982.752,  926.56 ,
        1062.88 , 1411.68 ],
       [ 301.024,  624.224, 1239.776, 1501.28 , 1559.008, 1220.96 , 1130.976,
        1179.488, 1349.984],
       [ 301.024,  624.224, 1239.776, 1524.32 , 1602.656, 1457.76 , 1414.88 ,
        1421.024, 1404.256],
       [ 411.232,  761.696, 1300.448, 1524.32 , 1602.656, 1513.824, 1524.96 ,
        1531.744, 1464.544],
       [ 556.512,  972.256, 1383.776, 1506.144, 1540.96 , 1513.824, 1472.352,
        1453.792, 1409.12 ],
       [ 609.76 , 1134.688, 1518.048, 1518.048, 1536.736, 1478.88 , 1472.352,
        1416.544, 1341.792]], dtype=float32)
Coordinates:
    band         int64 2
  * y            (y) float64 4.984e+06 4.962e+06 ... 4.828e+06 4.806e+06
  * x            (x) float64 5.217e+05 5.439e+05 ... 6.771e+05 6.993e+05
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    grid_mapping:  spatial_ref